# About

This notebook added an splendid `External Dataset` shared by [@mlconsult](https://www.kaggle.com/mlconsult) to `MLM` shared by [@tungmphung](https://www.kaggle.com/tungmphung). Thanks to everyone willing to share their knowledge, let's make the DS community better ;)

**Please check the original notebooks and discussion here:**
- [Matching + Baseline + MLM w/ Evaluation Metric](https://www.kaggle.com/chienhsianghung/matching-baseline-mlm-w-evaluation-metric)
- [Coleridge Initiative| Naive CV (Local Validation)](https://www.kaggle.com/chienhsianghung/coleridge-initiative-naive-cv-local-validation)
- [Evaluation metric implementation](https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230091)
- [[Coleridge] Predict with Masked Dataset Modeling](https://www.kaggle.com/tungmphung/coleridge-predict-with-masked-dataset-modeling)
- [score 57ish with additional govt datasets](https://www.kaggle.com/mlconsult/score-57ish-with-additional-govt-datasets)

**Previous results from:**
[Matching + Baseline + MLM w/ Evaluation Metric](https://www.kaggle.com/chienhsianghung/matching-baseline-mlm-w-evaluation-metric)

|   | CV | LB |
| --- | --- | --- |
| og | 0.604 | 0.536 |
| og s1.5 |   | 0.535 |
| og s2.5 |   | 0.535 |
| pred_mlm_labels | 0.108 |   |
| lables_list | 0.548 |   |
| all | 0.261 |   |
| og2all | 0.454 |   |
| 150 400 |   | 0.536 |
| 200 300 |   | 0.536 |
| 2021 |   | 0.536 |
| 42 |   | 0.536 |
| 42 s1.5 |   | 0.535 |
| 42 .70 |   | 0.536 |
| 42 .80 |   | 0.536 |
| MATCH_ONLY |   | 0.533 |
| BASELINE_HELPING |   | 0.536 |
| 1 |   | 0.536 |

# Setting

In [50]:
COMPUTE_CV =False #计算local cv
ALL_BLENDED = False
BASELINE_HELPING = False
MATCH_ONLY = False
KEN_MATCHING = True
BS_CLEANING = False

SEED = 42

# Import

In [101]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import AutoTokenizer, DataCollatorForLanguageModeling, \
AutoModelForMaskedLM, Trainer, TrainingArguments, pipeline

from typing import List
import string
from functools import partial

sns.set()
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if COMPUTE_CV: 
    print('this submission notebook will compute CV score but commit notebook will not')
else:
    print('this submission notebook will only be used to submit result')

this submission notebook will only be used to submit result


# Load data

In [153]:
train_path = './input/train.csv'
train_files_path = './input/train'
train = pd.read_csv(train_path)
from sklearn.model_selection import train_test_split

#扔上去
no_validation=False
COMPUTE_CV=False
if COMPUTE_CV:
    size_validation=0.005
    if no_validation:
        sample_submission=train[0:int(size_validation*len(train))]
    else:
        train,sample_submission = train_test_split(train,test_size=size_validation)
    print("len of train",len(train),"\nlen of sample_submission",len(sample_submission))    
    paper_test_folder = './input/train'
    test_files_path = paper_test_folder
else:
    sample_submission = pd.read_csv('./input/sample_submission.csv')
    paper_test_folder = './input/test'
    test_files_path = paper_test_folder
    
adnl_govt_labels_path = './input/data_set_800.csv'

In [154]:
papers = {}
for paper_id in tqdm(sample_submission['Id']):
    
    with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

  0%|          | 0/4 [00:00<?, ?it/s]

# Literal Matching

### Create a Knowledge Bank

In [155]:
all_labels = set()

for label_1, label_2, label_3 in train[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
    all_labels.add(str(label_1).lower())
    all_labels.add(str(label_2).lower())
    all_labels.add(str(label_3).lower())
    
print(f'No. different labels: {len(all_labels)}')

No. different labels: 180


#### Additional Govt Datasets

In [156]:
adnl_govt_labels = pd.read_csv(adnl_govt_labels_path)
govt_labels=set()
for l in adnl_govt_labels.title:
    all_labels.add(l)
    govt_labels.add(l)
print(f'No. different labels: {len(all_labels)}')

No. different labels: 2159


### Matching on test data

In [157]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()


def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

if not BS_CLEANING:
    def text_cleaning(text):
        '''
        Converts all text to lower case, Removes special charecters, emojis and multiple spaces
        text - Sentence that needs to be cleaned
        '''
        text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
        text = re.sub(' +', ' ', text)
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)
        return text
else:
    def text_cleaning(text):
        '''
        Converts all text to lower case, Removes special charecters, emojis and multiple spaces
        text - Sentence that needs to be cleaned
        '''
        text = ''.join([k for k in text if k not in string.punctuation]) #去除标点符号
        text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
        # text = re.sub("/'+/g", ' ', text)
        return text


def read_json_pub(filename, train_data_path=train_files_path, output='text'):
    json_path = os.path.join(train_data_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [188]:
#将datasets+缩写格式进行拆分
def unpackdatasets_abbre(s1):
    
    stop_list=["of","and","for","in","s"]
    sent=[]
    for s in s1.split():
        if s not in stop_list:
            sent.append(s)
    word=sent[-1]
    if  len(sent)-1!=len(word):
        return s1

    for i in range(len(sent)-1):
        if  sent[i][0]!=word[i]:
            return s1
    return [" ".join(sent[:-1]),word]
 


['alzheimer disease neuroimaging initiative', 'adni']

In [194]:
if not KEN_MATCHING:
    literal_preds = []
    for paper_id in tqdm(sample_submission['Id']):
        paper = papers[paper_id]
        text_1 = '. '.join(section['text'] for section in paper).lower()
        text_2 = totally_clean_text(text_1)
        
        labels = set()
        for label in all_labels:
            if label in text_1 or label in text_2:
                label=clean_text(label)
                label=unpackdatasets_abbre(label)
                if type(label)==list:
                    labels.add(clean_text(label[0]))
                    labels.add(clean_text(label[1]))
                else:
                    labels.add(clean_text(label))
        literal_preds.append('|'.join(labels))
    literal_preds[:5]

  0%|          | 0/4 [00:00<?, ?it/s]

#### Ken Matching

In [198]:
if KEN_MATCHING: #仅匹配额外数据集
    literal_preds = []
    for paper_id in tqdm(sample_submission['Id']):
        paper = papers[paper_id]
        text_1 = '. '.join(section['text'] for section in paper).lower()
        text_2 = totally_clean_text(text_1)
        
        labels = set()
        for label in govt_labels:
            if label in text_1 or label in text_2:
                label=clean_text(label)
                label=unpackdatasets_abbre(label)
                if type(label)==list:
                    labels.add(clean_text(label[0]))
                    labels.add(clean_text(label[1]))
                else:
                    labels.add(clean_text(label))
        literal_preds.append('|'.join(labels))
    literal_preds[:5]

  0%|          | 0/4 [00:00<?, ?it/s]

# Masked Dataset Modeling

### Paths and Hyperparameters

In [109]:
if not MATCH_ONLY:
    PRETRAINED_PATH = './input/archive/output-mlm/checkpoint-60000'
    TOKENIZER_PATH = './input/archive/model_tokenizer'

    MAX_LENGTH = 64
    OVERLAP = 20

    PREDICT_BATCH = 32 # a higher value requires higher GPU memory usage

    DATASET_SYMBOL = '$' # this symbol represents a dataset name
    NONDATA_SYMBOL = '#' # this symbol represents a non-dataset name

# Transform data to MLM format

### Load model and tokenizer

In [110]:
if not MATCH_ONLY:
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=True)
    model = AutoModelForMaskedLM.from_pretrained(PRETRAINED_PATH)

    mlm = pipeline(
        'fill-mask', 
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )

### Auxiliary functions

In [111]:
def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

def clean_paper_sentence(s):
    """
    This function is essentially clean_text without lowercasing.
    """
    s = re.sub('[^A-Za-z0-9]+', ' ', str(s)).strip()
    s = re.sub(' +', ' ', s)
    return s

def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

connection_tokens = {'s', 'of', 'and', 'in', 'on', 'for', 'data', 'dataset'}
def find_mask_candidates(sentence):
    """ 
    Extract masking candidates for Masked Dataset Modeling from a given $sentence.
    A candidate should be a continuous sequence of at least 2 words, 
    each of these words either has the first letter in uppercase or is one of
    the connection words ($connection_tokens). Furthermore, the connection 
    tokens are not allowed to appear at the beginning and the end of the
    sequence.
    """
    #定义的数据集特征
    def candidate_qualified(words): #
        while len(words) and words[0].lower() in connection_tokens:
            words = words[1:]
        while len(words) and words[-1].lower() in connection_tokens:
            words = words[:-1]
        
        return len(words) >= 2
    
    candidates = []
    
    phrase_start, phrase_end = -1, -1
    for id in range(1, len(sentence)):
        word = sentence[id]
        if word[0].isupper() or word in connection_tokens:
            if phrase_start == -1:
                phrase_start = phrase_end = id
            else:
                phrase_end = id
        else:
            if phrase_start != -1:
                if candidate_qualified(sentence[phrase_start:phrase_end+1]):
                    candidates.append((phrase_start, phrase_end))
                phrase_start = phrase_end = -1
    
    if phrase_start != -1:
        if candidate_qualified(sentence[phrase_start:phrase_end+1]):
            candidates.append((phrase_start, phrase_end))
    
    return candidates

### Transform 转换为MASK格式

In [112]:
if not MATCH_ONLY: 
    mask = mlm.tokenizer.mask_token
    all_test_data = []
    
    for paper_id in tqdm(sample_submission['Id']):
        # load paper
        paper = papers[paper_id]

        # extract sentences
        sentences = set([clean_paper_sentence(sentence) for section in paper 
                         for sentence in section['text'].split('.')
                        ])
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
        sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]
        sentences = [sentence.split() for sentence in sentences] # sentence = list of words

        # mask
        test_data = []
        for sentence in sentences:
            for phrase_start, phrase_end in find_mask_candidates(sentence):
                dt_point = sentence[:phrase_start] + [mask] + sentence[phrase_end+1:]
                test_data.append((' '.join(dt_point), ' '.join(sentence[phrase_start:phrase_end+1]))) # (masked text, phrase)

        all_test_data.append(test_data)

  0%|          | 0/99 [00:00<?, ?it/s]

### Predict

In [200]:
if not MATCH_ONLY:
    pred_mlm_labels = []

    for test_data in tqdm(all_test_data):
        pred_bag = set()

        if len(test_data):
            texts, phrases = list(zip(*test_data))
            mlm_pred = []
            for p_id in range(0, len(texts), PREDICT_BATCH):
                batch_texts = texts[p_id:p_id+PREDICT_BATCH]
                batch_pred = mlm(list(batch_texts), targets=[f' {DATASET_SYMBOL}', f' {NONDATA_SYMBOL}'])

                if len(batch_texts) == 1:
                    batch_pred = [batch_pred]

                mlm_pred.extend(batch_pred)

            for (result1, result2), phrase in zip(mlm_pred, phrases):
                if (result1['score'] > result2['score']*2 and result1['token_str'] == DATASET_SYMBOL) or\
                   (result2['score'] > result1['score']*2 and result2['token_str'] == NONDATA_SYMBOL):
                    pred_bag.add(clean_text(phrase))

        # filter labels by jaccard score 
        filtered_labels = []

        for label in sorted(pred_bag, key=len, reverse=True):
            if len(filtered_labels) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered_labels):
                label=clean_text(label)
                label=unpackdatasets_abbre(label)
                if type(label)==list:
                    filtered_labels.append(clean_text(label[0]))
                    filtered_labels.append(clean_text(label[1]))
                else:
                    filtered_labels.append(clean_text(label))
                
        pred_mlm_labels.append('|'.join(filtered_labels))
    
    pred_mlm_labels[:5]

  0%|          | 0/99 [00:00<?, ?it/s]

# Baseline Model

In [114]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data
    
    
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    # text = re.sub("/'+/g", ' ', text)
    return text

In [115]:
if BASELINE_HELPING or ALL_BLENDED:
    tqdm.pandas()

    train['text'] = train['Id'].progress_apply(read_append_return)

    sample_submission['text'] = sample_submission['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path))

    train.head()

In [116]:
if BASELINE_HELPING or ALL_BLENDED:
    tqdm.pandas()
    
    train['text'] = train['text'].progress_apply(text_cleaning)

## Aggregate final predictions and write submission file

In [33]:
final_predictions = []

if ALL_BLENDED:
    for literal_match, mlm_pred in zip(literal_preds, pred_mlm_labels):
        temp = [literal_match, mlm_pred]
        temp = [pred for pred in temp if pred]
        temp = ('|').join(temp)
        final_predictions.append(temp)
elif MATCH_ONLY:
    final_predictions = literal_preds

else:    
    for literal_match, mlm_pred in zip(literal_preds, pred_mlm_labels):
        if literal_match:
            final_predictions.append(literal_match)
        else:
            final_predictions.append(mlm_pred)

sample_submission['PredictionString'] = final_predictions

In [35]:
sample_submission['PredictionString'] = final_predictions
sample_submission[['Id', 'PredictionString']].to_csv('mlm_submission.csv', index=False)

sample_submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,adni|alzheimer s disease neuroimaging initiati...
1,2f392438-e215-4169-bebf-21ac4ff253e1,common core of data|nces common core of data|t...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,slosh model|noaa storm surge inundation|sea la...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,rural urban continuum codes


# Evaluation Metric

In [117]:
# https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230091
def compute_fbeta(y_true: List[List[str]],
                  y_pred: List[List[str]],
                  beta: float = 0.5) -> float:
    """Compute the Jaccard-based micro FBeta score.

    References
    ----------
    - https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation
    """

    def _jaccard_similarity(str1: str, str2: str) -> float:
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))

    tp = 0  # true positive
    fp = 0  # false positive
    fn = 0  # false negative
    for ground_truth_list, predicted_string_list in zip(y_true, y_pred):
        predicted_string_list_sorted = sorted(predicted_string_list)
        for ground_truth in sorted(ground_truth_list):            
            if len(predicted_string_list_sorted) == 0:
                fn += 1
            else:
                similarity_scores = [
                    _jaccard_similarity(ground_truth, predicted_string)
                    for predicted_string in predicted_string_list_sorted
                ]
                matched_idx = np.argmax(similarity_scores)
                if similarity_scores[matched_idx] >= 0.5:
                    predicted_string_list_sorted.pop(matched_idx)
                    tp += 1
                else:
                    fn += 1
        fp += len(predicted_string_list_sorted)

    tp *= (1 + beta ** 2)
    fn *= beta ** 2
    fbeta_score = tp / (tp + fp + fn)
    return fbeta_score

In [118]:
if COMPUTE_CV:
    for literal_match, mlm_pred in zip(literal_preds, pred_mlm_labels):
        if mlm_pred:
            literal_match=[literal_match.split("|")]
            mlm_pred=[mlm_pred.split("|")]
            if compute_fbeta(literal_match,mlm_pred)!=1:
                print(mlm_pred,"\n",literal_match,"\n")


[['alzheimer s disease neuroimaging initiative adni']] 
 [['adni', 'alzheimer s disease neuroimaging initiative adni']] 

[['german cnd', 'us adni']] 
 [['adni']] 

[['uw global data']] 
 [['ibtracs', 'international best track archive for climate stewardship']] 

[['alzheimer s disease neuroimaging initiative dataset', 'alzheimer s disease neuroimaging initiative adni']] 
 [['adni', 'alzheimer s disease neuroimaging initiative adni']] 

[['rush memory and aging project']] 
 [['adni']] 

[['alzheimer s disease neuroimaging initiative adni']] 
 [['adni', 'alzheimer s disease neuroimaging initiative adni', 'pubmed']] 

[['alzheimer s disease neuroimaging initiative adni']] 
 [['adni', 'baltimore longitudinal study of aging', 'baltimore longitudinal study of aging blsa', 'alzheimer s disease neuroimaging initiative adni']] 

[['environmental education database of organizations and programs eedop']] 
 [['common core of data', 'national center for education statistics common core of data']] 

In [37]:
if COMPUTE_CV:
    COMPUTE_CV_SCORE = compute_fbeta(sample_submission['cleaned_label'].apply(lambda x: [x]),\
                  sample_submission['PredictionString'].apply(lambda x: x.split('|')))
    print('COMPUTE_CV_SCORE =', COMPUTE_CV_SCORE)
else:
    print(f'COMPUTE_CV = {COMPUTE_CV}')
    
print(f'ALL_BLENDED = {ALL_BLENDED}')
print(f'BASELINE_HELPING = {BASELINE_HELPING}')
print(f'MATCH_ONLY = {MATCH_ONLY}')
print(f'KEN_MATCHING = {KEN_MATCHING}')
print(f'BS_CLEANING = {BS_CLEANING}')
print(f'SEED = {SEED}')

COMPUTE_CV = False
ALL_BLENDED = False
BASELINE_HELPING = False
MATCH_ONLY = False
KEN_MATCHING = True
BS_CLEANING = False
SEED = 42


|   | CV | LB |
| --- | --- | --- |
| s57 adnl_govt_labels |   | 0.573 |
| 42 w/ adnl_govt_labels |   | 0.557 |
| adnl_govt_labels_26897 | 0.136 |   |
| adnl_govt_labels |   | 0.556 |
| adnl_govt_labels KEN |   |   |
| 42 w/ adnl_govt_labels KEN |   | 0.557 |
| **RKM adnl_govt_labels w/ 42** | **0.514** | **0.574** |
| RKM adnl_govt_labels BS_CLEANING w/ 42 |   | 0.568 |
| RKM adnl_govt_labels_26897 |   | 0.244 |
| RKM adnl_govt_labels w/ 42_48 |   | 0.574 |
| **RKM adnl_govt_labels w/ 42_36** |   | **0.574** |
| RKM adnl_govt_labels w/ 42_60 |   | 0.574 |
| RKM adnl_govt_labels w/ 42_24 |   | 0.573 |
| RKM adnl_govt_labels w/ 42_60 |   |   |